In [1]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [2]:
def _parse_line(line):

    rx_dict = {
    'token': re.compile(r'var token = "(?P<token>.*)"\n'),
    'url': re.compile(r'var url = "(?P<url>.*)"\n'),
    'org': re.compile(r'var org = "(?P<org>.*)"\n'),
    'bucket': re.compile(r'var bucket = "(?P<bucket>.*)"\n'),
    }   

    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

filepath = '/root/flexi-pipe/config.go'
# open the file and read through it line by line
with open(filepath, 'r') as file_object:
    line = file_object.readline()
    while line:
        # at each line check for a match with a regex
        key, match = _parse_line(line)

        if key == 'token':
            token = match.group('token')
        elif key == 'url':
            url = match.group('url')
        elif key == 'org':
            org = match.group('org')
        elif key == 'bucket':
            bucket = match.group('bucket')
        
        line = file_object.readline()
# url="http://192.168.20.58:8086"
url = "http://localhost:8086"

In [31]:
start = 1696343921
end = 1705735851

node = "lotus"

In [22]:
def experiment(start_time, end_time, filepath):
    # Retrieve experiments data from csv
    data = pd.read_csv(filepath, header=None)
    df = pd.DataFrame(data)

    #Rename columns
    experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "parameter", 5: "d", 6: "dlo", 7: "dhi", 8: "dscore", 9: "dlazy", 10: "dout", 11: "gossipFactor", 12: "initialDelay", 13: "interval"}, errors='raise')

    #Correct timestamp
    experiments["start"] = experiments["start"].str.slice(0, 27)
    experiments["end"] = experiments["end"].str.slice(0, 27)

    #String to timestamp
    # experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    # experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
    experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


    experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
    experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

    #Drop fields we don't mneed for the moment
    exp = experiments.drop(columns=["runtime", "initialDelay"]).sort_values(by=["start"])

    #Get times for different intervals
    # intervals = exp["interval"].drop_duplicates().sort_values().reset_index(drop=True)
    # intervals.head(10)

    expTime = exp[exp['startUnix'].astype(int).between(start_time, end_time)]
    # expTime['experiment'] = expTime.index
    expTime = expTime.reset_index().rename({'index':'experiment'}, axis = 'columns')

    return expTime

experiments = experiment(start, end, '../../Datasets/experiments_raw.csv')
experiments.to_csv('../../Datasets/experiments_filtered.csv')
experiments.head(500)

/tmp/ipykernel_80161/2323312885.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
/tmp/ipykernel_80161/2323312885.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
0,383,2023-10-03 14:38:41.3311106,2023-10-03 15:08:41.3340502,unl,reference,8,6,12,4,8,2,0.25,1.00,1696343921,1696345721
1,384,2023-10-04 12:58:35.4458617,2023-10-04 13:13:35.4494771,unl,reference,8,6,12,4,8,2,0.25,1.00,1696424315,1696425215
2,385,2023-10-04 13:25:01.0752251,2023-10-04 13:40:01.0774462,unl,reference,8,6,12,4,8,2,0.25,1.00,1696425901,1696426801
3,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498,unl,reference,8,6,12,4,8,2,0.25,1.00,1696429128,1696430028
4,387,2023-10-04 15:38:33.0103914,2023-10-04 15:53:33.0161317,unl,reference,8,6,12,4,8,2,0.25,1.00,1696433913,1696434813
5,388,2023-10-04 16:20:43.6412122,2023-10-04 16:35:43.6457500,unl,reference,8,6,12,4,8,2,0.25,1.00,1696436443,1696437343
6,389,2023-10-04 17:00:47.8475357,2023-10-04 17:15:47.8502305,unl,reference,8,6,12,4,8,2,0.25,1.00,1696438847,1696439747
7,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,reference,8,6,12,4,8,2,0.25,1.00,1696445523,1696447323
8,391,2023-10-04 19:25:37.1533399,2023-10-04 19:55:37.1555856,unl,reference,8,6,12,4,8,2,0.25,1.00,1696447537,1696449337
9,392,2023-10-04 19:59:12.6589825,2023-10-04 20:29:12.6608307,unl,reference,8,6,12,4,8,2,0.25,1.00,1696449552,1696451352


In [23]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [24]:
def from_influx_count(url, token, org, start_time, end_time,grouping_key):
    # start_time = 1693222601 
    # end_time = 1693228430    
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)
        
        # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()
        
    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                       ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                        '|> filter(fn: (r) => r._measurement == "deliverMessage") '
                        '|> group(columns: ["_measurement", "_field"], mode: "by") '
                        '|> count()')
    client.close()

    if data_frame.empty:
        count = 0
    else:
        count = data_frame["_value"].min().astype(int)
    return count

In [25]:
#Validate data
validate = pd.DataFrame()

for index, row in experiments.iterrows():
    count = from_influx_count(url, token, org, row["startUnix"], row["endUnix"],"_measurement")
    d = {'experiment': [row['experiment']], 'count': [count], 'topology': [row["topology"]], 'd': [row["d"]],'dhi': [row["dhi"]],'dlo': [row["dlo"]],'dlazy': [row["dlazy"]],
        'dscore': [row["dscore"]],'dout': [row["dout"]],'gossipFactor': [row["gossipFactor"]],'interval': [row["interval"]],}
    aux = pd.DataFrame(data=d)
    validate = pd.concat([validate, aux])

# validate.head(200)

exps = experiments.merge(validate, on=['experiment', 'topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
exps = exps.loc[exps["count"]>0]

exps.head(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix,count
2,385,2023-10-04 13:25:01.0752251,2023-10-04 13:40:01.0774462,unl,reference,8,6,12,4,8,2,0.25,1.0,1696425901,1696426801,46733
3,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498,unl,reference,8,6,12,4,8,2,0.25,1.0,1696429128,1696430028,128631
6,389,2023-10-04 17:00:47.8475357,2023-10-04 17:15:47.8502305,unl,reference,8,6,12,4,8,2,0.25,1.0,1696438847,1696439747,18248
7,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,reference,8,6,12,4,8,2,0.25,1.0,1696445523,1696447323,344487
8,391,2023-10-04 19:25:37.1533399,2023-10-04 19:55:37.1555856,unl,reference,8,6,12,4,8,2,0.25,1.0,1696447537,1696449337,244016
9,392,2023-10-04 19:59:12.6589825,2023-10-04 20:29:12.6608307,unl,reference,8,6,12,4,8,2,0.25,1.0,1696449552,1696451352,130775
10,393,2023-10-04 20:32:47.3098616,2023-10-04 21:02:47.3149178,unl,informed,8,6,12,4,8,2,0.25,3.0,1696451567,1696453367,194668
11,394,2023-10-04 21:06:22.7755634,2023-10-04 21:36:22.7788800,unl,informed,8,6,12,4,8,2,0.25,3.0,1696453582,1696455382,165538
12,395,2023-10-04 21:39:58.6528165,2023-10-04 22:09:58.6545536,unl,informed,8,6,12,4,8,2,0.25,3.0,1696455598,1696457398,64772
13,396,2023-10-04 22:13:33.8146704,2023-10-04 22:43:33.8174094,unl,informed0,16,8,20,16,8,4,0.50,1.0,1696457613,1696459413,141092


In [26]:
#Remove fault executions

#get median
median = exps.drop(columns=['start','end','startUnix','endUnix', 'parameter', 'experiment']).drop_duplicates()
median = median.groupby(['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval']).agg({'count':['median', 'std', 'mean']})
median.columns = median.columns.droplevel(0)
median.reset_index(inplace=True)
median = median.loc[median['median'] > 0]

median.head(50)

,topology,d,dhi,dlo,dlazy,dscore,dout,gossipFactor,interval,median,std,mean
0,general,2,4,1,3,2,5,0.15,3.00,42318.0,59845.275319,42318.000000
1,general,3,5,2,2,2,2,0.50,1.00,63388.0,14596.972643,58323.666667
2,general,3,7,2,20,9,2,0.25,1.00,22025.5,28869.048556,22025.500000
3,general,4,7,2,9,10,2,0.50,1.00,62968.0,39320.030192,71566.333333
4,general,5,7,4,2,3,2,0.25,1.00,84236.0,33491.111970,95775.000000
5,general,5,20,3,9,7,4,0.50,1.00,179434.0,72252.912267,145625.000000
6,general,6,8,3,2,2,4,0.25,0.50,37796.5,42383.273358,37796.500000
7,general,6,8,4,2,2,2,0.25,1.00,62045.5,31217.900784,63183.833333
8,general,6,8,4,2,2,4,0.25,0.50,38482.0,19105.694839,27177.600000
9,general,6,8,4,2,2,4,0.25,3.00,11592.0,61427.235493,41103.333333


In [27]:
#Validate the data

validation = exps.merge(median, on=['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
validated = validation[(validation['count'] >= validation['mean']-(0.15*validation['std']))]
validated = validated.drop(columns=['startUnix', 'endUnix'])
validated.dropna()

validated.head(100)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,count,median,std,mean
3,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,reference,8,6,12,4,8,2,0.25,1.00,344487,142423.0,149514.670084,208016.555556
4,391,2023-10-04 19:25:37.1533399,2023-10-04 19:55:37.1555856,unl,reference,8,6,12,4,8,2,0.25,1.00,244016,142423.0,149514.670084,208016.555556
7,505,2023-11-07 18:12:32.6142390,2023-11-07 18:42:32.6167070,unl,reference,8,6,12,4,8,2,0.25,1.00,408150,142423.0,149514.670084,208016.555556
8,506,2023-11-07 18:46:01.6133827,2023-11-07 19:16:01.6158393,unl,reference,8,6,12,4,8,2,0.25,1.00,408686,142423.0,149514.670084,208016.555556
9,393,2023-10-04 20:32:47.3098616,2023-10-04 21:02:47.3149178,unl,informed,8,6,12,4,8,2,0.25,3.00,194668,162285.0,90760.806400,175292.500000
10,394,2023-10-04 21:06:22.7755634,2023-10-04 21:36:22.7788800,unl,informed,8,6,12,4,8,2,0.25,3.00,165538,162285.0,90760.806400,175292.500000
12,507,2023-11-07 19:19:31.9016456,2023-11-07 19:49:31.9049682,unl,informed,8,6,12,4,8,2,0.25,3.00,337252,162285.0,90760.806400,175292.500000
15,396,2023-10-04 22:13:33.8146704,2023-10-04 22:43:33.8174094,unl,informed0,16,8,20,16,8,4,0.50,1.00,141092,131035.5,74389.509136,106159.333333
16,397,2023-10-04 22:47:09.3837110,2023-10-04 23:17:09.3868442,unl,informed0,16,8,20,16,8,4,0.50,1.00,125279,131035.5,74389.509136,106159.333333
18,510,2023-11-07 21:00:03.5598397,2023-11-07 21:30:03.5622902,unl,informed0,16,8,20,16,8,4,0.50,1.00,136792,131035.5,74389.509136,106159.333333


In [62]:
final = validated.drop(columns=['start','end', 'parameter', 'count', 'std', 'median', 'mean']).drop_duplicates()
final['n_nodes'] = 24
final = final.reset_index()#.rename({'index':'experiment'}, axis = 'columns')
final.head(100)

,index,experiment,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes
0,3,390,unl,8,6,12,4,8,2,0.25,1.00,24
1,4,391,unl,8,6,12,4,8,2,0.25,1.00,24
2,7,505,unl,8,6,12,4,8,2,0.25,1.00,24
3,8,506,unl,8,6,12,4,8,2,0.25,1.00,24
4,9,393,unl,8,6,12,4,8,2,0.25,3.00,24
5,10,394,unl,8,6,12,4,8,2,0.25,3.00,24
6,12,507,unl,8,6,12,4,8,2,0.25,3.00,24
7,15,396,unl,16,8,20,16,8,4,0.50,1.00,24
8,16,397,unl,16,8,20,16,8,4,0.50,1.00,24
9,18,510,unl,16,8,20,16,8,4,0.50,1.00,24


In [29]:
#Limit extraction to 15 minutes

experiments = validated.drop(columns=['count','median', 'mean','std'])#.rename(columns={'start':'originalStart','end':'originalEnd'})

# experiments['originalStart'] = pd.to_datetime(experiments["originalStart"], format='mixed')
# experiments['originalEnd'] = pd.to_datetime(experiments["originalEnd"], format='mixed')

# experiments['start'] = experiments['originalStart'] + pd.Timedelta(hours=0, minutes=7, seconds=30)
# experiments['end'] = experiments['originalEnd'] - pd.Timedelta(hours=0, minutes=7, seconds=30)

experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed").astype('int64') / 10**9
experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed").astype('int64') / 10**9

experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

experiments.to_csv('../../Datasets/experiments_valid.csv')
experiments.head(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
3,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,reference,8,6,12,4,8,2,0.25,1.0,1696445523,1696447323
4,391,2023-10-04 19:25:37.1533399,2023-10-04 19:55:37.1555856,unl,reference,8,6,12,4,8,2,0.25,1.0,1696447537,1696449337
7,505,2023-11-07 18:12:32.6142390,2023-11-07 18:42:32.6167070,unl,reference,8,6,12,4,8,2,0.25,1.0,1699380752,1699382552
8,506,2023-11-07 18:46:01.6133827,2023-11-07 19:16:01.6158393,unl,reference,8,6,12,4,8,2,0.25,1.0,1699382761,1699384561
9,393,2023-10-04 20:32:47.3098616,2023-10-04 21:02:47.3149178,unl,informed,8,6,12,4,8,2,0.25,3.0,1696451567,1696453367
10,394,2023-10-04 21:06:22.7755634,2023-10-04 21:36:22.7788800,unl,informed,8,6,12,4,8,2,0.25,3.0,1696453582,1696455382
12,507,2023-11-07 19:19:31.9016456,2023-11-07 19:49:31.9049682,unl,informed,8,6,12,4,8,2,0.25,3.0,1699384771,1699386571
15,396,2023-10-04 22:13:33.8146704,2023-10-04 22:43:33.8174094,unl,informed0,16,8,20,16,8,4,0.50,1.0,1696457613,1696459413
16,397,2023-10-04 22:47:09.3837110,2023-10-04 23:17:09.3868442,unl,informed0,16,8,20,16,8,4,0.50,1.0,1696459629,1696461429
18,510,2023-11-07 21:00:03.5598397,2023-11-07 21:30:03.5622902,unl,informed0,16,8,20,16,8,4,0.50,1.0,1699390803,1699392603


In [57]:
def from_influx_message(url, token, org, start_time, end_time):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "message" and r.node == "'+node+'" and r._field == "type" and (r._value == 11 or r._value == 12 or r._value == 3 or r._value == 2)) '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')

    client.close()
    
    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', 'type']].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [60]:
query_mess = from_influx_message(url, token, org, 1696438847, 1696439747)

query_mess.head(10)

KeyError: "None of [Index(['_time', 'type'], dtype='object')] are in the [columns]"

In [ ]:
# message = pd.DataFrame()
execs = experiments#.drop(columns=['originalStart', 'originalEnd'])
# final=final.loc[final["identifier"] == 1]

for idx, exec in execs.iterrows():
    print(exec["startUnix"], exec["endUnix"])
    query_mess = from_influx_message(url, token, org, exec["startUnix"], exec["endUnix"])

    query_mess["experiment"] = exec["experiment"]

    query_mess["start"] = exec["start"]
    query_mess["end"] = exec["end"]
            
    query_mess["_time"] = pd.to_datetime(query_mess["_time"])
        # message = pd.concat([message,query_mess])

    query_mess.to_csv('../../Datasets/partialTables/mess_'+node+'_'+str(exec["experiment"])+'.csv')
        
    del query_mess
    gc.collect()


1696445523 1696447323


In [ ]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    if data_frame.empty == False:
        data_frame.reset_index(inplace=True)
        df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
        df["_time"] = pd.to_datetime(df["_time"])
    else:
        df = data_frame

    return df

In [59]:
gossip = pd.DataFrame()
execs = experiments.drop(columns=['originalStart', 'originalEnd'])

for index, row in final.iterrows():
    # print(row["start"], row["end"], row["experiment"])
    execs = exps.loc[(exps["topology"] == row["topology"]) & (exps["d"] == row["d"]) &
                            (exps["dlo"] == row["dlo"]) & (exps["dhi"] == row["dhi"]) &
                            (exps["dscore"] == row["dscore"]) & (exps["dlazy"] == row["dlazy"]) &
                            (exps["dout"] == row["dout"]) & (exps["gossipFactor"] == row["gossipFactor"]) &                    
                            (exps["interval"] == row["interval"])]

    for idx, exec in execs.iterrows():
        # print(exec["startUnix"], exec["endUnix"])
        query_iwant = from_influx(url, token, org, "iwant", exec["startUnix"], exec["endUnix"],"_measurement")

        if query_iwant.empty == False:
            query_iwant["identifier"] = row["identifier"]
            query_iwant["experiment"] = exec["experiment"]
 
            query_iwant["_time"] = pd.to_datetime(query_iwant["_time"])
            gossip = pd.concat([gossip, query_iwant])

            query_iwant.to_csv('./iwant_'+str(exec["experiment"])+'.csv')z

            del query_iwant

        query_ihave = from_influx(url, token, org, "ihave", exec["startUnix"], exec["endUnix"],"_measurement")
        if query_ihave.empty == False:
            query_ihave["identifier"] = row["identifier"]
            query_ihave["experiment"] = exec["experiment"]

            query_ihave["_time"] = pd.to_datetime(query_ihave["_time"])
            gossip = pd.concat([gossip, query_ihave])

            query_ihave.to_csv('./ihave_'+str(exec["experiment"])+'.csv')
        
            del query_ihave

        gc.collect()

gossip.head(10)

,_time,_measurement,identifier,experiment
0,2023-11-21 15:24:53.032623+00:00,iwant,0,642
1,2023-11-21 15:40:26.891377+00:00,iwant,0,642
0,2023-11-21 15:24:52.988402+00:00,ihave,0,642
1,2023-11-21 15:24:52.989674+00:00,ihave,0,642
2,2023-11-21 15:24:52.990178+00:00,ihave,0,642
3,2023-11-21 15:24:52.990617+00:00,ihave,0,642
4,2023-11-21 15:24:52.992429+00:00,ihave,0,642
5,2023-11-21 15:24:52.992519+00:00,ihave,0,642
6,2023-11-21 15:24:52.993070+00:00,ihave,0,642
7,2023-11-21 15:24:52.993105+00:00,ihave,0,642


In [60]:
gossip.to_csv('./gossipTimeSeries.csv', mode='a')

In [11]:
data = pd.read_csv('../exp_filtered.csv', header=0,  index_col=0)
exps = pd.DataFrame(data)

exps.head(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix,count
0,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498,unl,reference,8,6,12,4,8,2,0.25,1.0,1696429128,1696430028,128631
3,389,2023-10-04 17:00:47.8475357,2023-10-04 17:15:47.8502305,unl,reference,8,6,12,4,8,2,0.25,1.0,1696438847,1696439747,18248
4,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,reference,8,6,12,4,8,2,0.25,1.0,1696445523,1696447323,344487
5,391,2023-10-04 19:25:37.1533399,2023-10-04 19:55:37.1555856,unl,reference,8,6,12,4,8,2,0.25,1.0,1696447537,1696449337,244016
6,392,2023-10-04 19:59:12.6589825,2023-10-04 20:29:12.6608307,unl,reference,8,6,12,4,8,2,0.25,1.0,1696449552,1696451352,130775
7,393,2023-10-04 20:32:47.3098616,2023-10-04 21:02:47.3149178,unl,informed,8,6,12,4,8,2,0.25,3.0,1696451567,1696453367,194668
8,394,2023-10-04 21:06:22.7755634,2023-10-04 21:36:22.7788800,unl,informed,8,6,12,4,8,2,0.25,3.0,1696453582,1696455382,165538
9,395,2023-10-04 21:39:58.6528165,2023-10-04 22:09:58.6545536,unl,informed,8,6,12,4,8,2,0.25,3.0,1696455598,1696457398,64772
10,396,2023-10-04 22:13:33.8146704,2023-10-04 22:43:33.8174094,unl,informed0,16,8,20,16,8,4,0.50,1.0,1696457613,1696459413,141092
11,397,2023-10-04 22:47:09.3837110,2023-10-04 23:17:09.3868442,unl,informed0,16,8,20,16,8,4,0.50,1.0,1696459629,1696461429,125279


In [12]:
#Bandwidth
# recvRpc = pd.DataFrame()
# deliverMessage = pd.DataFrame()
# publishMessage = pd.DataFrame()

message = pd.DataFrame()


for idx, exec in exps.iterrows():
    query_rpc = from_influx(url, token, org, "recvRPC", exec["startUnix"], exec["endUnix"],"_measurement")
    
    query_rpc["experiment"] = exec["experiment"]
    
    query_rpc["start"] = exec["start"]
    query_rpc["end"] = exec["end"]
            
    query_rpc["_time"] = pd.to_datetime(query_rpc["_time"])
    
    query_rpc.to_csv('./rpc_'+str(exec["experiment"])+'.csv')
        
    message = pd.concat([message,query_rpc])
        

message.head(10)

,_time,_measurement,experiment,start,end
0,2023-10-04 14:20:46.195636+00:00,recvRPC,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498
1,2023-10-04 14:20:46.195995+00:00,recvRPC,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498
2,2023-10-04 14:20:46.232999+00:00,recvRPC,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498
3,2023-10-04 14:20:46.233042+00:00,recvRPC,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498
4,2023-10-04 14:20:46.233092+00:00,recvRPC,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498
5,2023-10-04 14:20:46.233541+00:00,recvRPC,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498
6,2023-10-04 14:20:46.273808+00:00,recvRPC,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498
7,2023-10-04 14:20:46.280621+00:00,recvRPC,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498
8,2023-10-04 14:20:46.281112+00:00,recvRPC,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498
9,2023-10-04 14:20:46.281933+00:00,recvRPC,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498


In [14]:
message = message.drop(columns=['start', 'end'])
                       
message.reset_index(inplace=True)
message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)
by_time = message.groupby([message['experiment'],pd.Grouper(key="_time", freq='10s')])['index'].agg('count').reset_index()
by_time.head(100)

/tmp/ipykernel_83365/2211971587.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,index
0,386,2023-10-04 14:20:40+00:00,6869
1,386,2023-10-04 14:20:50+00:00,45535
2,386,2023-10-04 14:21:00+00:00,68173
3,386,2023-10-04 14:21:10+00:00,35262
4,386,2023-10-04 14:21:20+00:00,38928
5,386,2023-10-04 14:21:30+00:00,28143
6,386,2023-10-04 14:21:40+00:00,26485
7,386,2023-10-04 14:21:50+00:00,30122
8,386,2023-10-04 14:22:00+00:00,22359
9,386,2023-10-04 14:22:10+00:00,19063


In [16]:
by_time=by_time.rename(columns={'index':'rpc'})
by_time.to_csv('./rpc.csv')